In [ ]:
import os
import numpy as np
import pandas as pd
# import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
# import sklearn.metrics
# import scipy.special

# import plotly.graph_objects as go
# import matplotlib.pyplot as plt
import gc
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.models
import h5py
# import datetime

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
HEIGHT = 137
WIDTH = 236
# From kaggle
ROOT_DIR = '/kaggle/input/'
# local
# ROOT_DIR = './Data/bengaliai-cv19/'

In [ ]:
model_fname = ROOT_DIR+'models/model-2020-01-09-06-03'

json_file = open(model_fname+'.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(model_fname+'.hdf5')
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(optimizer=keras.optimizers.Adam(),
#     loss=customLoss,
    loss = keras.losses.binary_crossentropy,
    metrics=['binary_accuracy',keras.metrics.Recall()])


In [ ]:
test = pd.read_csv(ROOT_DIR+'bengaliai-cv19/test.csv')
test_files = [str.format(ROOT_DIR+'bengaliai-cv19/test_image_data_{0}.parquet',i) for i in range(4) ]
submission = pd.read_csv(ROOT_DIR+'bengaliai-cv19/sample_submission.csv')

In [ ]:
offset = 0
for fname in test_files:
    data = pd.read_parquet(fname)
    
    print('loading file')
    chunk_images = np.expand_dims(
        data.iloc[:,1:].values.astype(np.float16).reshape(-1, HEIGHT, WIDTH)/255.0,
        axis=3)
    
    chunk_files_count = chunk_images.shape[0]
    print(str(chunk_files_count)+' loaded')
    
    predictions_chunk = loaded_model.predict(chunk_images)
    print ('predicted ' +  str(predictions_chunk.shape))

    chunk_vowel_diacritic =  predictions_chunk[:,0:11].argmax(axis=1)
    chunk_grapheme_root = predictions_chunk[:,11:179].argmax(axis=1)
    chunk_consonant_diacritic = predictions_chunk[:,179:186].argmax(axis=1)
    
    idxes = [3*i+offset for i in range(chunk_files_count)]
    submission.loc[:,'target'].iloc[idxes]=chunk_consonant_diacritic

    idxes = [1+3*i+offset for i in range(chunk_files_count)]
    submission.loc[:,'target'].iloc[idxes]=chunk_grapheme_root

    idxes = [2+3*i+offset for i in range(chunk_files_count)]
    submission.loc[:,'target'].iloc[idxes]=chunk_vowel_diacritic
    
    offset+=3*chunk_files_count
    print ('transformed '+str(offset))

In [ ]:
submission.to_csv('submission.csv',index=False)